In [5]:
#for this cell:
#input the training data, cutting it's elements into parts, labeled "trainfile" to differentiate from testfile

trainfile=open("/Users/daniaalma/Desktop/NLP Homework/Homework 1/berp-POS-training.txt").read().split('\n')
from collections import Counter, defaultdict

trainlines=[]

for line in trainfile:
    if not line.strip():
        continue
    trainlines.append(line)

#don't make the wordlist, taglist or indexlist here, just get max tags from the training data    
#different name for the word tag counts so we can separate the variable names, this is only for
#getting the maximum word tag count for definition

wordtagcounts = defaultdict(Counter)
for line in trainlines:         
    if not line.strip():
        continue          
    index, word, tags = line.split()
    wordtagcounts[word.lower()][tags] += 1
    
stats = wordtagcounts

pos_tags = {word: stats[word].most_common(1)[0][0] for word in stats}

with open ('/Users/daniaalma/Desktop/NLP Homework/Homework 1/newtestfile.txt','w') as newtestfile:
    for line in trainlines:    
        if not line.strip():
            continue          
        index, word, tags = line.split()
        if index =="1":
            startline=("\t".join(['0','<s>','<s>'])+"\n")
            newtestfile.write(startline + "\t".join([index, word, tags])+"\n")
        else:
            if word == '.':
                newtestfile.write("\t".join([index, '</s>','</s>'])+"\n")
            else:
                newtestfile.write("\t".join([index, word, tags])+"\n")
            
#reprint the file onto a text file to test the accuracy and get tags, this seems like baseline
            

In [6]:
index=[]        
word=[]

with open('/Users/daniaalma/Desktop/NLP Homework/Homework 1/berp-POS-training.txt','r') as file:
    for line in file.readlines():
        if not len(line.strip())==0:
            index.append (line.split('\t')[0]) 
            word.append(line.split('\t')[1])

with open('/Users/daniaalma/Desktop/NLP Homework/Homework 1/output.txt','w') as writefile: 
      for x, y in zip(index,word):
        writefile.write("\t".join([x, y, pos_tags[y]])+"\n")
        if y=='.':
            writefile.write('\n')

In [2]:
sent=[]
testlines=[]
linesinfile=[]

testfile=open("/Users/daniaalma/Desktop/NLP Homework/Homework 1/assgn2testset.txt").read().split('\n')

for line in testfile:         
    if not line.strip():
        continue

for line in testfile:
    if not line.strip():
        continue
    index,word= line.split()
    if index == "1":
        sent=['<s>', word]
        testlines.append(sent)
    else:
        if word == '.':
            sent.append('</s>')
        else:
            sent.append(word)
            

#del testlines[0]

In [ ]:
#UNK words code, removing low frequency words
newtestfile=open("/Users/daniaalma/Desktop/NLP Homework/Homework 1/newtestfile.txt").read().split('\n')
#training not test file
indexlist=[]
wordlist=[]
taglist=[]

#get wordlist,taglist,indexlist 

word_tag_counts = defaultdict(int)
for line in newtestfile:         
    if not line.strip():
        continue
    index, word, tags = line.split()
    word_tag_counts[(word.lower(),tags)] += 1
    indexlist.append(index)
    wordlist.append(word)
    taglist.append(tags)
    
taglist=set(taglist)
taglist.remove(":")
taglist.remove(".")
   
wordcount = {} #unkify
for word in wordlist:
    if word in wordcount:
        wordcount[word] += 1
    else:
        wordcount[word] = 1
        
word_tag_list=list(word_tag_counts.keys())
print(word_tag_counts)
#UNK_wordtagcounts=defaultdict(int)
UNK_wordamount=0
UNK_words=[]#unkify_me

for key, value in wordcount.items():
    if value <3:
        UNK_words.append(key)
        #UNK_wordamount += value
        #UNK_wordtagcounts [('<UNK>', key[1])] +=1   

        
        #sent.insert(0, "0\t<s>t")
#UNK_wordtagcounts [('<UNK>', 'WDT')]=2

import pandas as pd
#i need this 
#get elements to do emission

emission_probabilities={}
emission={}

for line in newtestfile:         
    if not line.strip():
        continue
    element=line.strip().split("\t")
    if element[1] in UNK_words:
        if '<UNK>' in emission_probabilities.keys():
            if element[2] in emission_probabilities['<UNK>'].keys():
                emission_probabilities['<UNK>'][element[2]] += 1
            else:
                emission_probabilities['<UNK>'][element[2]] = 1
        else:
            emission_probabilities['<UNK>'] = {element[2]:1}
    else:
        if element[1] in emission_probabilities.keys():
            if element[2] in emission_probabilities[element[1]].keys():
                emission_probabilities[element[1]][element[2]] += 1
            else:
                emission_probabilities[element[1]][element[2]] = 1
        else:
            emission_probabilities[element[1]] = {element[2]:1}
#print(emission_probabilities)            
emission=pd.DataFrame(emission_probabilities).fillna(0)

In [ ]:
for word in emission.columns:
    total=emission[word].sum()
    for pos in emission.index:
        emission.loc[pos,word]=float(emission.loc[pos,word])/float(total)
        
#print(emission)

In [ ]:
emission

In [1]:
#bigram probabilities from training data

def readData():

    tag=[]
    with open('/Users/daniaalma/Desktop/NLP Homework/Homework 1/newtestfile.txt','r') as file:
        for line in file.readlines():

            if not len(line.strip())==0:
                tag.append(line.split('\t')[2].strip())
    return tag
#opening file, get data, read

def createBigram(tag):
    bigramList=[]
    bigramcounts={}
    unigramcounts={} 
    
    for i in range(len(tag)):
        if i<len(tag)-1:
            if not tag[i]=='.\n':
                bigramList.append((tag[i],tag[i+1]))
            
                if (tag[i], tag[i+1]) in bigramcounts:
                    bigramcounts[(tag[i], tag[i+1])] +=1
                else:
                    bigramcounts[(tag[i], tag[i+1])] =1
                
        if tag[i] in unigramcounts:
            unigramcounts[tag[i]] +=1
            
        else:
            unigramcounts[tag[i]]=1
            
    return bigramList, unigramcounts, bigramcounts

#defining the bigram count

def calculatebigramprobability(bigramList,unigramcounts,bigramcounts):
    
    transition={}
 
    for bigram in bigramList:
        tag1=bigram[0]
        if bigram not in bigramcounts:
            print(bigram)
            print(bigramcounts)
        if tag1 not in unigramcounts:
            print(tag1)
        transition[bigram]=(bigramcounts.get(bigram)+0.01)/(unigramcounts.get(tag1))#p(wi|wi-1)=(c(wiwi-1))/(c(wi-1))
        
        #smoothing +.01
        #if you add tags here then this is the transition


    return transition

In [2]:
#transition, we gucci
tag=readData()
bigramList, unigramcounts, bigramcounts = createBigram(tag)
transition=calculatebigramprobability(bigramList,unigramcounts,bigramcounts)


NameError: name 'log' is not defined

In [ ]:
#viterbi
 
def viterbi (transition, emission, sentence):
    #set tag as start of sentence and assign a value
    
    tags=taglist
    taglist.remove

    for t in tags:
        for t2 in tags:
            if (t2,t) not in transition:
                transition[(t2,t)]=0.000001
    
    start={}
    for tag in tags:
        if tag == '<s>':
            start['<s>']=1
        else:
            start[tag] = 0
   
    i=0
    for word in sentence:
        if word not in list(emission):
            sentence[i]= '<UNK>'
        i = i+1         
    vit=[{}]

    for tag in tags: 

        vit[0][tag]={"current":start[tag]*emission.loc[(tag,sentence[0])], "previous":None}

    for index in range (1,len(sentence)):
        vit.append({})
        for t in tags:

            (prob, oldtag)=max((vit[index-1][prev]["current"]*transition[(prev,t)]*emission.loc[(t,sentence[index])],prev)for prev in tags)
            vit[index][t]={"current":prob,"previous":oldtag}

    likeliest_tags=[]
    maxprob=max(value["current"]for value in vit[-1].values())
    previous= None
    for tag, everything in vit[-1].items():
        if everything["current"]==maxprob:
            likeliest_tags.append(tags)
            previous=tag
            break

    for t in range(len(vit)-2,-1,-1):
        likeliest_tags.insert(0, vit[t+1][previous]["previous"])
        previous=vit[t+1][previous]["previous"]
        
        
    return likeliest_tags
    
    

In [ ]:
with open('/Users/daniaalma/Desktop/NLP Homework/Homework 1/Elmadhun-Dania-assgn2-test-output.txt', 'a') as output:
    for sentence in testlines:
        copyofsentence=sentence[:]
        print(copyofsentence)
        #making a copy of the sentence list so we don't edit the sentence itself
        sentence_indices=range(len(sentence))
        #make an index of the sentence length
        likeliest_tags=viterbi(transition, emission, sentence)
        
        for index, word, tag in zip(sentence_indices, copyofsentence, likeliest_tags):
            if word == '<s>':
                continue
            elif word == '</s>':
                output.write("{}\t{}\t{}\n".format(index,'.','.'))
            else:
                output.write("{}\t{}\t{}\n".format(index,word,tag))
                print(index,word,tag)
        output.write("\n")
    
output.close()

